In [1]:
import json
import pandas as pd
import seaborn as sns
import numpy as np

# 1. Wiki docs 불러오기

In [2]:
json_path = 'opt/ml/input/data/wikipedia_documents.json'
with open(json_path) as wiki_file:
    wiki_dict = json.load(wiki_file)

# 2. Wiki docs column들 살펴보기

<br>
먼저, wiki에 text, title, document_id 외의 column들은 어떤 정보를 담고 있을지 각 column들을 살펴보기로 했습니다.

In [3]:
df = pd.DataFrame.from_dict(wiki_dict, orient='index')

In [4]:
df

,text,corpus_source,url,domain,title,author,html,document_id
0,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",위키피디아,TODO,None,나라 목록,None,None,0
1,이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정...,위키피디아,TODO,None,나라 목록,None,None,1
2,현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친...,위키피디아,TODO,None,백남준,None,None,2
3,"아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로...",위키피디아,TODO,None,아오조라 문고,None,None,3
4,저자 사망 이후 50년이 지나 저작권이 소멸한 메이지 시대부터 쇼와 시대 초기까지의...,위키피디아,TODO,None,아오조라 문고,None,None,4
...,...,...,...,...,...,...,...,...
60608,"오키나와 현립 박물관·미술관( , Okinawa Prefectural Museum ...",위키피디아,None,None,오키나와 현립 박물관·미술관,None,None,60608
60609,"1936년 7월, 오키나와현 교육위원회 부설 향토 박물관(沖縄県教育会附設郷土博物館)...",위키피디아,None,None,오키나와 현립 박물관·미술관,None,None,60609
60610,박물관은 개관 10주년이되는 2017년에 관의 애칭 및 마스코트를 일반인에게 공모했...,위키피디아,None,None,오키나와 현립 박물관·미술관,None,None,60610
60611,단결권 및 단체교섭권 협약(Right to Organise and Collectiv...,위키피디아,None,None,단결권 및 단체교섭권 협약,None,None,60611


`'corpus_source'` column은 '위키피디아'라는 값만을 가지고 있습니다. 특이사항이 없습니다.

In [5]:
df.groupby('corpus_source')['document_id'].nunique()

corpus_source
위키피디아    60613
Name: document_id, dtype: int64

<br>

`'url'` column은 'TODO'라는 값이 4554개이고, 나머지는 None인 것으로 보입니다. 0행부터는 TODO를 값으로 가지다가 뒤로 가면 None을 가지는 것으로 보이니, 4554행 이후로 살펴볼 필요가 있겠습니다.

In [6]:
df.groupby('url')['document_id'].nunique()

url
TODO    4554
Name: document_id, dtype: int64

<br>

`'domain'`, `'author'`, `'html'` column들은 비어있습니다. 무시해도 되는 column입니다.

In [7]:
df.groupby('domain')['document_id'].nunique()

Series([], Name: document_id, dtype: int64)

In [8]:
df.groupby('author')['document_id'].nunique()

Series([], Name: document_id, dtype: int64)

In [9]:
df.groupby('html')['document_id'].nunique()

Series([], Name: document_id, dtype: int64)

# 3. url column 값 바탕으로, 4554행 기준으로 나누어 EDA

<br>
0부터 5행까지 url이 TODO값을 가지는 것을 볼 수 있습니다.

In [10]:
df[:5]

,text,corpus_source,url,domain,title,author,html,document_id
0,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",위키피디아,TODO,None,나라 목록,None,None,0
1,이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정...,위키피디아,TODO,None,나라 목록,None,None,1
2,현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친...,위키피디아,TODO,None,백남준,None,None,2
3,"아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로...",위키피디아,TODO,None,아오조라 문고,None,None,3
4,저자 사망 이후 50년이 지나 저작권이 소멸한 메이지 시대부터 쇼와 시대 초기까지의...,위키피디아,TODO,None,아오조라 문고,None,None,4


<br>
4554행부터 살펴보니, 0행부터의 값이 중복되는 것을 확인할 수 있습니다. 그렇다면 정말 0행-4553행의 모든 값이 중복인지 살펴보겠습니다.

In [11]:
df[4554:4559]

,text,corpus_source,url,domain,title,author,html,document_id
4554,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",위키피디아,None,None,나라 목록,None,None,4554
4555,이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정...,위키피디아,None,None,나라 목록,None,None,4555
4556,현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친...,위키피디아,None,None,백남준,None,None,4556
4557,"아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로...",위키피디아,None,None,아오조라 문고,None,None,4557
4558,저자 사망 이후 50년이 지나 저작권이 소멸한 메이지 시대부터 쇼와 시대 초기까지의...,위키피디아,None,None,아오조라 문고,None,None,4558


text의 중복 판단을 위해 fuzzywuzzy 함수를 사용하였습니다.

In [12]:
from fuzzywuzzy import fuzz

non_duplication = []

for i in range(4554):
    if (df.iloc[i]['title'] == df.iloc[i + 4554]['title']) and (fuzz.ratio(df.iloc[i]['text'], df.iloc[i + 4554]['text']) > 95):
        pass
    else:
        non_duplication.append(df.iloc[i]['document_id'])

non_duplication

[2460]

fuzzywuzzy ratio를 95%로 했을 때, 2460을 제외하고 모든 text가 중복입니다. 2460을 살펴보겠습니다.

In [13]:
print(df.iloc[2460]['text'], '\n', df.iloc[2460 + 4554]['text'])

일반적으로 PvP에서 해적계열과 백병전특화계열의 직업이 포격계열 보다 불리하기 때문에 백병계열군의 직업군은 인기가 없다.

다만, 일방적으로 약한 상대를 공격하는 PK(예를 들어 유저해적의 강습의 경우)와, 수탈로만 가능한 아이템을 얻기 위해서는 이러한 직업이 상대에게나 NPC에게 더 많은 피해를 주며, 백병전을 통해서만 수탈이 가능하기 때문에 유저해적들이나 수탈을 목적으로 하는 플레이어들은 이들 계열 직업을 선택한다.

그러나 예전에는 해적계열직업과 백병전특화계열의 직업은 대체적으로 포격계직업에 비해서 포격전에서 열세에 있었으나 요즘은 유료아이템으로 강화한 대포를 사용함으로써 이러한 불리함이 역전되어 원거리에서는 포격, 근거리에서는 백병전을 자유롭게 구사할 수 있게 되었다. 그리고 FS조선(자유조선)업데이트로 인하여 선박 전용함스킬이나 선박 강화스킬에서 직격저지(스킬 사용시 선루를 포격으로 맞아도 크리티컬이 뜨지않음)라는 선박스킬이 생김으로써 백병스킬의 인기는 더욱더 올라가게 되었다.

* 수습군인: 제독과 해적 등 전투계 직업의 수습. 적 함대와 싸워 현상금과 전리품을 얻는다.
* 용병: 각국으로부터 의뢰를 받고 싸우는 전사. 근거리포격이나 백병전에 뛰어나다.
* 준사관: 유격을 담당하는 해사함장. 원거리 포격을 잘한다.
* 경호원: 위생을 생업으로 한다. 방어력과 회복력에 뛰어나다.
* 토박이 해적: 타국의 함대를 덮쳐 전리품을 빼앗는다. 갤리계 배의 조종을 잘 한다.
* 해군사관: 충분히 공적을 인정받은 사관. 해상전 전반에 능통하다.
* 사략해적: 나라의 보증을 얻은 해적. 타국의 함대를 용서없이 덮친다.
* 현상금 사냥꾼: 해적의 토벌을 가업으로 하는 직업. 항해술과 전투력이 뛰어나다.
* 조선공: 배의 수리와 조선을 하는 직업. 배의 취급에도 뛰어나다.
* 군의: 선원의 치료를 전문으로 하는 직업.
* 상급사관: 세계의 바다를 둘러보고 해적이나 적국의 함대를 무찌르는 함장.
* 전술가: 다채로운 전술에 정통한 해상전의 프로. 무기나 배의 생산도 소

2460도 사실상 같은 text인 것을 확인할 수 있습니다. 이 두 text의 fuzzywuzzy ratio를 살펴보겠습니다.

In [14]:
fuzz.ratio(df.iloc[2460]['text'], df.iloc[2460 + 4554]['text'])

89

In [15]:
print(len(df.iloc[2460]['text']), len(df.iloc[2460 + 4554]['text']))

1490 1273


fuzzywuzzy ratio의 limit을 정하기 위해, 아예 다른 문서들에서는 어느 정도 나오는지를 살펴보겠습니다.

In [16]:
fuzz.ratio(df.iloc[4554]['text'], df.iloc[4556]['text'])

10

아예 다른 text에서는 현저히 낮은 점수인 10이 나옵니다. 향후 중복을 판단하기 위해 너무 높은 점수를 사용하지 않아도 될 것 같아, 50을 사용해서 진행해 보겠습니다. 

# 4. 나머지 문서 중복 확인

우선, 0부터 4553까지는 모두 중복되는 문서들이므로 제거가 가능합니다. 

In [17]:
df2 = df[4554:]
df2 = df2.reset_index()
df2 = df2.drop('index', axis=1)
df2

,text,corpus_source,url,domain,title,author,html,document_id
0,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",위키피디아,None,None,나라 목록,None,None,4554
1,이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정...,위키피디아,None,None,나라 목록,None,None,4555
2,현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친...,위키피디아,None,None,백남준,None,None,4556
3,"아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로...",위키피디아,None,None,아오조라 문고,None,None,4557
4,저자 사망 이후 50년이 지나 저작권이 소멸한 메이지 시대부터 쇼와 시대 초기까지의...,위키피디아,None,None,아오조라 문고,None,None,4558
...,...,...,...,...,...,...,...,...
56054,"오키나와 현립 박물관·미술관( , Okinawa Prefectural Museum ...",위키피디아,None,None,오키나와 현립 박물관·미술관,None,None,60608
56055,"1936년 7월, 오키나와현 교육위원회 부설 향토 박물관(沖縄県教育会附設郷土博物館)...",위키피디아,None,None,오키나와 현립 박물관·미술관,None,None,60609
56056,박물관은 개관 10주년이되는 2017년에 관의 애칭 및 마스코트를 일반인에게 공모했...,위키피디아,None,None,오키나와 현립 박물관·미술관,None,None,60610
56057,단결권 및 단체교섭권 협약(Right to Organise and Collectiv...,위키피디아,None,None,단결권 및 단체교섭권 협약,None,None,60611


먼저, 나머지 문서들 중 text가 정확히 일치하는 문서가 있는지 알아보면, 아직도 82개의 중복된 문서가 존재합니다.

In [18]:
len(df2['text']) - len(dict.fromkeys(df2['text']))

82

중복된 문서들을 파악하기 위해, dictionary를 만들어서 문서를 count해보면 다음과 같습니다.

In [19]:
text_dict = dict.fromkeys(df2['text'])

for i, text in enumerate(df2['text']):
    if text_dict[text]:
        text_dict[text] += 1
    else:
        text_dict[text] = 1

In [20]:
filtered_dict = dict(filter(lambda elem: elem[1] >=2, text_dict.items()))
len(filtered_dict)

35

In [21]:
filtered_dict

{"19세기 말부터 러시아의 진보적 귀족과 러시아 농촌 내 농민에게서 자생적으로 발생한 브 나로드에서 파생된 나로드니키주의(Народничество) ― 소위 당시 인민주의라고도 불린 ― 에 대하여 레닌은 농업 발전의 모든 양상을 거부한 극단적인 정체적 사고라고 비판하였다. 당시 러시아 인민주의자들은 자본주의적 농업 소유 구조는 농업 소유 구조에서 완전한 악(惡)이라고 보았다. 이들은 자본주의적 농업 소유 구조는 인간의 선의에서 벗어난 반인류적 구조인 것이며, 과거의 원시적 농업 형태가 선(善)에 해당하는 공산주의적 공동체의 전형이라고 주장했다. 블라디미르 레닌은 당시 유럽국가의 산업 발전과 농업 구조의 변혁을 실증적으로 검토하였고, 마르크스주의의 지대 이론에 따라 자본주의적 농업 구조는 이전 봉건적 농업 구조에 대해서 발전된 양상이라고 보았다. 자본주의 특유의 소규모 자영농 위주의 농업 생산 구조는 산업의 발전을 촉진시킨 결과물로 나온 것이며, 기본적으로 이 과정에서 지주의 권한을 대폭 약화시켰다. 즉, 이전의 봉건적 농업 구조에 비해 '농지'(農地)이라는 생산수단에 대해서 인민이 더 쉽게 접근할 수 있게 된다고 본 것이다. 이러한 입장에 기반하여 블라디미르 레닌은 러시아 인민주의자들이 경제 문제에 도덕주의라는 허울을 들씌웠다고 비판하였다. 즉, 그들의 주장은 고대사회부터 현재에 이르기까지 진행된 모든 소유 구조의 발전 과정을 염두에 두지 않은 주장이란 것이다. 블라디미르 레닌에게 있어서 이 인민주의 운동은 일종의 공상적 사회주의와 비슷한 것이었다.": 2,
 '당시 교조주의 입장에서 농업 문제를 바라봤던 율리우스 마르토프(Юлий Мартов, 1873 - 1923)와 레프 트로츠키를 비롯한 멘셰비키 일원들은 블라디미르 레닌의 강령을 두고 \'농본주의\'(Аграризм)라고 비판하였다. 멘셰비키들은 농업은 중세봉건제 사회에서 자본주의 사회로 넘어오는 과도기적 단계에서 그 발전성을 완전히 다했다고 보았다. 즉, 사회주의로의 이행은 완전히 산업노동자의 수중에

35개의 중복되는 문서들이 어떤 식으로 중복이 발생하고 있는지 확인해 보도록 하겠습니다.

In [22]:
_duplicated_text_filter = df2['text'].isin(filtered_dict)

In [23]:
# 35: original document의 수, 82: 그 외 중복되는 document의 수, 117: original documents + 중복 documents 
df_duplicated_texts = df2[_duplicated_text_filter]
df_duplicated_texts.sort_values(by='text')

,text,corpus_source,url,domain,title,author,html,document_id
45232,"10대 닥터\n \n*TARDISODE 8\n*""The beast and his a...",위키피디아,None,None,The Impossible Planet,None,None,49786
45287,"10대 닥터\n \n*TARDISODE 8\n*""The beast and his a...",위키피디아,None,None,The Satan Pit,None,None,49841
1213,19세기 말부터 러시아의 진보적 귀족과 러시아 농촌 내 농민에게서 자생적으로 발생한...,위키피디아,None,None,레닌주의,None,None,5767
55709,19세기 말부터 러시아의 진보적 귀족과 러시아 농촌 내 농민에게서 자생적으로 발생한...,위키피디아,None,None,노농동맹,None,None,60263
51830,2000년부터 김정일은 독립채산제로 인한 자체 수익 사업을 권장하며 방위산업이든 일...,위키피디아,None,None,89호집적회로공장,None,None,56384
...,...,...,...,...,...,...,...,...
38383,파일:Pierre Trudeau (1975).jpg|캐나다 캐나다<br />피에르 ...,위키피디아,None,None,제4회 G7 정상회담,None,None,42937
1302,한단이 함락되고 공자 가가 대군으로 쫓겨들어온 후에 얼마 지나지 않아 진나라군은 역...,위키피디아,None,None,진나라,None,None,5856
1316,한단이 함락되고 공자 가가 대군으로 쫓겨들어온 후에 얼마 지나지 않아 진나라군은 역...,위키피디아,None,None,진나라,None,None,5870
52445,한산도함은 국내 군함 중 최대 규모의 의무실과 병상을 갖췄다. 독도함의 1.8배의 ...,위키피디아,None,None,ATH-81 한산도,None,None,56999


'레닌주의'와 '노농농맹'의 경우, 두 개념이 비슷해서인지 같은 소단락이 들어가 있었고, '헤미스 국립공원'과 '카라코람 야생 동물보호구역'의 경우 다른 공원임에도 불구하고 같은 소단락이 사용된 것을 확인할 수 있었습니다. 주로 text 기반 분석이 이루어 지므로, 이 경우 text를 중심으로 title을 concat하고, 하나의 document id만을 남기는 것이 좋을 것 같습니다. document_id는 ground_truth 확인 용으로 사용되므로, dataset 내에 존재하는 document_id 중심으로 남기는 방향으로 하겠습니다.

# 5. dataset과의 비교

In [24]:
import os.path as p
from datasets import load_from_disk, load_dataset, load_metric

In [25]:
train_dataset = load_from_disk(p.join('/opt/ml/input/data/', 'train_dataset'))

In [26]:
train_dataset

DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 240
    })
})

In [27]:
dummy_dataset = load_from_disk(p.join('/opt/ml/input/data/', 'dummy_dataset'))

In [28]:
dummy_dataset

DatasetDict({
    train: Dataset({
        features: ['answers', 'context', 'id', 'question', 'title'],
        num_rows: 200
    })
    validation: Dataset({
        features: ['answers', 'context', 'id', 'question', 'title'],
        num_rows: 20
    })
})

In [29]:
dummy_dataset['train'][0]

{'answers': {'answer_start': [223], 'text': ['기원전 90년']},
 'context': '루키우스의 초반 생애는 알려진 것이 없다. 그가 처음으로 역사에 이름이 오른 것은 기원전 100년에 포푸라레스(민중파)의 호민관 루키우스 아풀레이우스 사투르니누스의 몰락에 관여한 것이었다. 기원전 94년에 법무관으로 선출되었지만, 재무관과 조영관 취임 이력은 없었던 것 같다. 법무관 임기 이후 마케도니아 속주의 총독으로 부임했다.\\n\\n기원전 91년에 발발한 동맹시 전쟁이 로마를 뒤덮은 가운데, 기원전 90년에 루키우스는 집정관에 선출되어 삼니움인을 격파했다. 또한 그 해에 동맹시 전쟁에 가담하지 않은 동맹 도시들에게 로마 시민권을 부여하는 내용의 법안(율리우스 법)을 통과시켰다. 법률의 성립에 의해 동맹시 전쟁은 잦아들었고, 기원전 88년에 전쟁은 완전히 종결되었다. 또한 89년에 감찰관이 된 루키우스는 새로 로마 시민이 된 시민들의 선거구를 제정하는 것도 결정했다.\\n\\n그러나 기원전 87년에 가이우스 마리우스의 당파가 이탈리아 본국에 군사를 보내 침공하여 로마를 점령하면서 옵티마테스(벌족파)에 속하는 것으로 간주된 루키우스와 동생 카이사르 워피스쿠스는 시가전이 전개된 끝에 마리우스파에 의해 살해되었다. 2명의 수급은 원로원의 연단에 놓였다고 티투스 리비우스는 전하고 있다.\\n\\n루키우스는 같은 이름의 루키우스 카이사르와 율리아, 2명의 아이들이 알려져 있다. 또한 나중에 종신독재관이 된 율리우스 카이사르는 루키우스의 조카에 해당한다.',
 'id': 'mrc-1-000711',
 'question': '루키우스가 율리우스 법을 통과시킨 시기는?',
 'title': '루키우스 율리우스 카이사르'}

dummy_dataset에는 document_id가 없으므로, train_dataset만 확인하도록 하겠습니다.

In [30]:
dataset_doc_ids = train_dataset['train']['document_id'] + train_dataset['validation']['document_id']

In [31]:
dataset_doc_ids = list(dict.fromkeys(dataset_doc_ids))
len(dataset_doc_ids)

3504

3504개의 document들이 사용된 것을 확인할 수 있습니다. 우선 document id들을 정렬한 후, 중복으로 파악되는 document가 얼마나 있는지 알아보도록 하겠습니다.

In [32]:
dataset_doc_ids.sort()
dataset_doc_ids[:5]

[4572, 4591, 4600, 4616, 4628]

우선, 4554 이전의 모든 document들이 중복으로 파악되었는데 dataset에서 사용되는 document id들은 4572부터 시작하므로, 중복으로 파악된 문서들은 포함하지 않는다는 것을 확인할 수 있습니다.

In [33]:
duplications = []
for k in dataset_doc_ids:
    if k in list(df_duplicated_texts['document_id']):
        duplications.append(k)
duplications

[60263]

# 6. 데이터 정제

다행히도(?) 우리 데이터셋에서 중복되는 문서는 한 개만 보입니다. 나머지 문서를 이 문서 중심으로 합쳐 보겠습니다.

In [34]:
overlapped_texts = []
for i in duplications:
    overlapped_texts.append(df.iloc[i]['text'])

overlapped_texts

["19세기 말부터 러시아의 진보적 귀족과 러시아 농촌 내 농민에게서 자생적으로 발생한 브 나로드에서 파생된 나로드니키주의(Народничество) ― 소위 당시 인민주의라고도 불린 ― 에 대하여 레닌은 농업 발전의 모든 양상을 거부한 극단적인 정체적 사고라고 비판하였다. 당시 러시아 인민주의자들은 자본주의적 농업 소유 구조는 농업 소유 구조에서 완전한 악(惡)이라고 보았다. 이들은 자본주의적 농업 소유 구조는 인간의 선의에서 벗어난 반인류적 구조인 것이며, 과거의 원시적 농업 형태가 선(善)에 해당하는 공산주의적 공동체의 전형이라고 주장했다. 블라디미르 레닌은 당시 유럽국가의 산업 발전과 농업 구조의 변혁을 실증적으로 검토하였고, 마르크스주의의 지대 이론에 따라 자본주의적 농업 구조는 이전 봉건적 농업 구조에 대해서 발전된 양상이라고 보았다. 자본주의 특유의 소규모 자영농 위주의 농업 생산 구조는 산업의 발전을 촉진시킨 결과물로 나온 것이며, 기본적으로 이 과정에서 지주의 권한을 대폭 약화시켰다. 즉, 이전의 봉건적 농업 구조에 비해 '농지'(農地)이라는 생산수단에 대해서 인민이 더 쉽게 접근할 수 있게 된다고 본 것이다. 이러한 입장에 기반하여 블라디미르 레닌은 러시아 인민주의자들이 경제 문제에 도덕주의라는 허울을 들씌웠다고 비판하였다. 즉, 그들의 주장은 고대사회부터 현재에 이르기까지 진행된 모든 소유 구조의 발전 과정을 염두에 두지 않은 주장이란 것이다. 블라디미르 레닌에게 있어서 이 인민주의 운동은 일종의 공상적 사회주의와 비슷한 것이었다."]

In [35]:
_filter = df_duplicated_texts['text'].isin(overlapped_texts)
df_overlapped = df_duplicated_texts[_filter]
df_overlapped

,text,corpus_source,url,domain,title,author,html,document_id
1213,19세기 말부터 러시아의 진보적 귀족과 러시아 농촌 내 농민에게서 자생적으로 발생한...,위키피디아,None,None,레닌주의,None,None,5767
55709,19세기 말부터 러시아의 진보적 귀족과 러시아 농촌 내 농민에게서 자생적으로 발생한...,위키피디아,None,None,노농동맹,None,None,60263


In [36]:
# 60263번을 중심으로 document_ids를 합치도록 하겠습니다.

df2.loc[55709,'title'] = df2.loc[1213, 'title'] + ', ' + df2.loc[55709, 'title']
df2.loc[55709,:]

text             19세기 말부터 러시아의 진보적 귀족과 러시아 농촌 내 농민에게서 자생적으로 발생한...
corpus_source                                                위키피디아
url                                                           None
domain                                                        None
title                                                   레닌주의, 노농동맹
author                                                        None
html                                                          None
document_id                                                  60263
Name: 55709, dtype: object

In [37]:
df3 = df2.drop([1213])
df3

,text,corpus_source,url,domain,title,author,html,document_id
0,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",위키피디아,None,None,나라 목록,None,None,4554
1,이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정...,위키피디아,None,None,나라 목록,None,None,4555
2,현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친...,위키피디아,None,None,백남준,None,None,4556
3,"아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로...",위키피디아,None,None,아오조라 문고,None,None,4557
4,저자 사망 이후 50년이 지나 저작권이 소멸한 메이지 시대부터 쇼와 시대 초기까지의...,위키피디아,None,None,아오조라 문고,None,None,4558
...,...,...,...,...,...,...,...,...
56054,"오키나와 현립 박물관·미술관( , Okinawa Prefectural Museum ...",위키피디아,None,None,오키나와 현립 박물관·미술관,None,None,60608
56055,"1936년 7월, 오키나와현 교육위원회 부설 향토 박물관(沖縄県教育会附設郷土博物館)...",위키피디아,None,None,오키나와 현립 박물관·미술관,None,None,60609
56056,박물관은 개관 10주년이되는 2017년에 관의 애칭 및 마스코트를 일반인에게 공모했...,위키피디아,None,None,오키나와 현립 박물관·미술관,None,None,60610
56057,단결권 및 단체교섭권 협약(Right to Organise and Collectiv...,위키피디아,None,None,단결권 및 단체교섭권 협약,None,None,60611


이제 나머지 문서에서 중복을 먼저 제거한 후, 중복 문서들은 가장 작은 document_id를 중심으로 title을 합치는 방식으로 병합하도록 하겠습니다.
이 때, 원본 document_id 정보를 남겨두기 위해 document_ids라는 column을 추가했습니다.

In [38]:
_remove_handled_data_filter = df_duplicated_texts['document_id'].isin(list(df_overlapped['document_id']))
df_duplicated_texts_2 = df_duplicated_texts[~_remove_handled_data_filter]
df_duplicated_texts_2 = df_duplicated_texts_2.sort_values(by='document_id')

In [39]:
_duplicate_filter = df3['text'].isin(list(df_duplicated_texts['text']))
df_non_duplicates = df3[~_duplicate_filter]
document_ids = []
for doc_id in list(df_non_duplicates['document_id']):
    document_ids.append([doc_id])

In [40]:
df_non_duplicates['document_ids'] = document_ids
df_non_duplicates

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,text,corpus_source,url,domain,title,author,html,document_id,document_ids
0,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",위키피디아,None,None,나라 목록,None,None,4554,[4554]
1,이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정...,위키피디아,None,None,나라 목록,None,None,4555,[4555]
2,현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친...,위키피디아,None,None,백남준,None,None,4556,[4556]
3,"아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로...",위키피디아,None,None,아오조라 문고,None,None,4557,[4557]
4,저자 사망 이후 50년이 지나 저작권이 소멸한 메이지 시대부터 쇼와 시대 초기까지의...,위키피디아,None,None,아오조라 문고,None,None,4558,[4558]
...,...,...,...,...,...,...,...,...,...
56054,"오키나와 현립 박물관·미술관( , Okinawa Prefectural Museum ...",위키피디아,None,None,오키나와 현립 박물관·미술관,None,None,60608,[60608]
56055,"1936년 7월, 오키나와현 교육위원회 부설 향토 박물관(沖縄県教育会附設郷土博物館)...",위키피디아,None,None,오키나와 현립 박물관·미술관,None,None,60609,[60609]
56056,박물관은 개관 10주년이되는 2017년에 관의 애칭 및 마스코트를 일반인에게 공모했...,위키피디아,None,None,오키나와 현립 박물관·미술관,None,None,60610,[60610]
56057,단결권 및 단체교섭권 협약(Right to Organise and Collectiv...,위키피디아,None,None,단결권 및 단체교섭권 협약,None,None,60611,[60611]


In [41]:
duplicated_dict = dict.fromkeys(df_duplicated_texts_2['text'])
for i, text in enumerate(df_duplicated_texts_2['text']):
    if duplicated_dict[text]:
        duplicated_dict[text]['title'] += (', ' + df_duplicated_texts_2.iloc[i]['title'])
        duplicated_dict[text]['document_ids'].append( df_duplicated_texts_2.iloc[i]['document_id'])
    else:
        duplicated_dict[text] = {'corpus_source': df_duplicated_texts_2.iloc[i]['corpus_source'],
                                 'url': df_duplicated_texts_2.iloc[i]['url'],
                                 'domain': df_duplicated_texts_2.iloc[i]['domain'],
                                 'title':  df_duplicated_texts_2.iloc[i]['title'],
                                 'author': df_duplicated_texts_2.iloc[i]['author'],
                                 'html': df_duplicated_texts_2.iloc[i]['html'],
                                 'document_id': df_duplicated_texts_2.iloc[i]['document_id'], 
                                 'document_ids': [ df_duplicated_texts_2.iloc[i]['document_id']]}

df_duplicated_processed = pd.DataFrame(duplicated_dict)
df_duplicated_processed = df_duplicated_processed.transpose().reset_index()
df_duplicated_processed.columns = ['text','corpus_source', 'url', 'domain', 'title', 'author', 'html', 'document_id', 'document_ids']
df_duplicated_processed

,text,corpus_source,url,domain,title,author,html,document_id,document_ids
0,당시 교조주의 입장에서 농업 문제를 바라봤던 율리우스 마르토프(Юлий Мартов...,위키피디아,None,None,"레닌주의, 노농동맹",None,None,5768,"[5768, 60264]"
1,"진은 춘추시대에 들어가는 동시에 제후가 되었지만, 풍속적으로는 중원 제국(諸國)과 ...",위키피디아,None,None,"진나라, 진나라",None,None,5849,"[5849, 5863]"
2,"기원전 236년, 진나라는 조나라(趙)를 본격적으로 공략하기 전에 이사의 계략을 써...",위키피디아,None,None,"진나라, 진나라",None,None,5851,"[5851, 5865]"
3,조나라를 멸망시키기 위해 수십만을 동원한 수차례의 원정이 조나라의 저항에 막혀버리자...,위키피디아,None,None,"진나라, 진나라",None,None,5854,"[5854, 5868]"
4,4차 침공 이후 진나라 군대는 3년의 휴식기간을 가졌다. 그동안 진나라는 가만히 있...,위키피디아,None,None,"진나라, 진나라",None,None,5855,"[5855, 5869]"
5,한단이 함락되고 공자 가가 대군으로 쫓겨들어온 후에 얼마 지나지 않아 진나라군은 역...,위키피디아,None,None,"진나라, 진나라",None,None,5856,"[5856, 5870]"
6,"진왕 정 21년, 기원전 226년 진나라군은 연나라의 수도 계(薊)를 점령하고 북방...",위키피디아,None,None,"진나라, 진나라",None,None,5857,"[5857, 5871]"
7,"진왕 정 21년, 기원전 226년 진왕 정은 신정에서 일어난 반란을 무자비하게 진압...",위키피디아,None,None,"진나라, 진나라",None,None,5858,"[5858, 5872]"
8,"기원전 214년 경, 진 시황제는 10만 여 명의 대군을 보내 내 북쪽의 국경을 지...",위키피디아,None,None,"진나라, 진나라",None,None,5860,"[5860, 5874]"
9,진시황은 6국을 정복하여 전국시대를 종식시키고 초기에 36군을 설치하였으며 지방관을...,위키피디아,None,None,"진나라, 진나라",None,None,5862,"[5862, 5876]"


In [42]:
new_wiki = pd.concat([df_non_duplicates, df_duplicated_processed]).sort_values(by='document_id')
new_wiki = new_wiki.reset_index()
new_wiki = new_wiki.drop('index', axis=1)

# 8. json으로 새로운 wiki 문서 저장

In [96]:
new_wiki.to_json('/opt/ml/input/data/wikipedia_documents_processed.json', orient='index')

In [97]:
json_path = 'opt/ml/input/data/wikipedia_documents_processed.json'
with open(json_path) as wiki_file:
    wiki_dict2 = json.load(wiki_file)

In [98]:
df_new = pd.DataFrame.from_dict(wiki_dict2, orient='index')

In [99]:
df_new

,text,corpus_source,url,domain,title,author,html,document_id,document_ids
0,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",위키피디아,None,None,나라 목록,None,None,4554,[4554]
1,이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정...,위키피디아,None,None,나라 목록,None,None,4555,[4555]
2,현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친...,위키피디아,None,None,백남준,None,None,4556,[4556]
3,"아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로...",위키피디아,None,None,아오조라 문고,None,None,4557,[4557]
4,저자 사망 이후 50년이 지나 저작권이 소멸한 메이지 시대부터 쇼와 시대 초기까지의...,위키피디아,None,None,아오조라 문고,None,None,4558,[4558]
...,...,...,...,...,...,...,...,...,...
55971,"오키나와 현립 박물관·미술관( , Okinawa Prefectural Museum ...",위키피디아,None,None,오키나와 현립 박물관·미술관,None,None,60608,[60608]
55972,"1936년 7월, 오키나와현 교육위원회 부설 향토 박물관(沖縄県教育会附設郷土博物館)...",위키피디아,None,None,오키나와 현립 박물관·미술관,None,None,60609,[60609]
55973,박물관은 개관 10주년이되는 2017년에 관의 애칭 및 마스코트를 일반인에게 공모했...,위키피디아,None,None,오키나와 현립 박물관·미술관,None,None,60610,[60610]
55974,단결권 및 단체교섭권 협약(Right to Organise and Collectiv...,위키피디아,None,None,단결권 및 단체교섭권 협약,None,None,60611,[60611]
